# 🪑 FreeSpot - Table Occupancy Detection API

## Running Backend with YOLO on Google Colab

Notebook ini akan menjalankan backend FastAPI dengan YOLO detection di Google Colab.

### Setup Steps:
1. Jalankan semua cell secara berurutan
2. Copy public URL yang muncul di output Cell 5
3. Gunakan URL tersebut di frontend React Anda

---

## 📦 Cell 1: Install Dependencies

In [ ]:
!pip install -q fastapi uvicorn python-multipart opencv-python-headless opencv-contrib-python-headless ultralytics nest-asyncio pyngrok sse-starlette
print("✅ All dependencies installed!")
print("📦 Installed packages:")
print("   - FastAPI: Web framework")
print("   - OpenCV: Video processing with RTSP support")
print("   - YOLO: Object detection")
print("   - Ngrok: Public URL tunneling")
print("   - SSE: Server-Sent Events")

## 📚 Cell 2: Import Libraries

In [ ]:
from fastapi import FastAPI, File, UploadFile, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse, JSONResponse, FileResponse
from sse_starlette.sse import EventSourceResponse
import cv2
import numpy as np
from ultralytics import YOLO
import io
import json
from typing import List, Dict
import asyncio
from datetime import datetime
import time
import os
import uuid
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()
print("✅ Libraries imported successfully!")


## 🔑 Cell 3: Setup Ngrok (Optional)

Untuk mendapatkan public URL yang stabil:
1. Daftar di https://dashboard.ngrok.com/signup
2. Copy authtoken dari dashboard
3. Uncomment dan ganti YOUR_TOKEN dengan token Anda

In [ ]:
# Uncomment baris di bawah dan ganti dengan token Anda
# ngrok.set_auth_token("YOUR_NGROK_TOKEN_HERE")
print("⚠️  Ngrok token not set. You'll get a temporary URL.")
print("   For stable URL, sign up at: https://dashboard.ngrok.com/")

## 🤖 Cell 3.5: Download YOLO Model (Optional)

**Jalankan cell ini jika ingin pre-download model YOLO**

In [ ]:
# Pre-download YOLO model
from ultralytics import YOLO
import os

print("📥 Downloading YOLO model...")
print("Choose your model size:")
print("   - yolov11n.pt: Nano (fastest, smallest)")
print("   - yolov11s.pt: Small (balanced)")
print("   - yolov11m.pt: Medium (more accurate, slower)")
print("   - yolov11l.pt: Large (most accurate, slowest)")
print()

# Download model - pilih salah satu:
MODEL_NAME = "yolov11x.pt"  # Ubah sesuai kebutuhan

print(f"⏳ Downloading {MODEL_NAME}...")
model = YOLO(MODEL_NAME)
print(f"✅ Model {MODEL_NAME} downloaded successfully!")
print(f"📍 Saved to: {os.path.abspath(MODEL_NAME)}")

# Test model
print("\n🧪 Testing model...")
results = model.predict(source="https://ultralytics.com/images/bus.jpg", save=False)
print(f"✅ Model test successful! Detected {len(results[0].boxes)} objects")

## ⚙️ Cell 4: Application Code & Configuration (Betav1.0 Logic)

In [ ]:
app = FastAPI(title="FreeSpot API", description="Table Occupancy Detection with YOLO")

# CORS Configuration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

print("📦 Loading YOLO model...")
model = YOLO('yolo11x.pt')
print("✅ YOLO model loaded!")

# ========================================
# KONFIGURASI MEJA - SESUAIKAN DENGAN LAYOUT ANDA!
# Format: [x_min, y_min, x_max, y_max]
# ========================================
TABLES = [
    {"id": 1, "name": "Meja 1", "coords": [50, 100, 200, 250]},
    {"id": 2, "name": "Meja 2", "coords": [250, 100, 400, 250]},
    {"id": 3, "name": "Meja 3", "coords": [450, 100, 600, 250]},
    {"id": 4, "name": "Meja 4", "coords": [50, 300, 200, 450]},
    {"id": 5, "name": "Meja 5", "coords": [250, 300, 400, 450]},
    {"id": 6, "name": "Meja 6", "coords": [450, 300, 600, 450]},
]

PROXIMITY_THRESHOLD = 100  # Jarak threshold dalam pixel

# ========================================
# ADVANCED DETECTION CONFIGURATION
# ========================================
CONFIDENCE_THRESHOLD = 0.6  # Confidence minimal untuk deteksi (60%)
MIN_PERSON_AREA = 1000  # Minimum luas person (filter noise)
MAX_PERSON_AREA = 500000  # Maximum luas person (filter false positive)

# Storage untuk video hasil deteksi
processed_videos = {}

def filter_person_detections(boxes) -> List:
    """Filter person detections untuk akurasi lebih tinggi"""
    valid_persons = []
    for box in boxes:
        if int(box.cls[0]) != 0:  # Skip non-person
            continue

        coords = box.xyxy[0].cpu().numpy()
        confidence = float(box.conf[0])

        # Filter by confidence
        if confidence < CONFIDENCE_THRESHOLD:
            continue

        # Calculate area
        width = coords[2] - coords[0]
        height = coords[3] - coords[1]
        area = width * height

        # Filter by area
        if area < MIN_PERSON_AREA or area > MAX_PERSON_AREA:
            continue

        # Filter by aspect ratio (person harus lebih tinggi dari lebar)
        aspect_ratio = height / width if width > 0 else 0
        if aspect_ratio < 0.5 or aspect_ratio > 5:
            continue

        valid_persons.append({
            'coords': coords.tolist(),
            'confidence': confidence,
            'area': area
        })
    return valid_persons

def calculate_iou(box1, box2):
    """Calculate Intersection over Union"""
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    if x2_inter < x1_inter or y2_inter < y1_inter:
        return 0.0

    inter_area = (x2_inter - x1_inter) * (y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0.0

def calculate_distance(person_box, table_coords):
    """Hitung jarak dengan multi-point (lebih akurat)"""
    # Center point
    person_center_x = (person_box[0] + person_box[2]) / 2
    person_center_y = (person_box[1] + person_box[3]) / 2

    # Bottom-center (kaki, lebih akurat untuk orang duduk)
    person_bottom_x = person_center_x
    person_bottom_y = person_box[3]

    table_center_x = (table_coords[0] + table_coords[2]) / 2
    table_center_y = (table_coords[1] + table_coords[3]) / 2

    # Distance dari bottom person ke center table
    distance_bottom = np.sqrt(
        (person_bottom_x - table_center_x) ** 2 +
        (person_bottom_y - table_center_y) ** 2
    )

    # Distance dari center person
    distance_center = np.sqrt(
        (person_center_x - table_center_x) ** 2 +
        (person_center_y - table_center_y) ** 2
    )

    return min(distance_bottom, distance_center)

def check_table_occupancy(persons: List, tables: List[Dict]) -> Dict:
    """Check occupancy dengan IoU dan proximity (lebih akurat)"""
    table_status = {}

    for table in tables:
        table_id = table["id"]
        table_coords = table["coords"]
        is_occupied = False
        closest_distance = float('inf')
        detection_method = "none"

        # Expand table area (margin 20%)
        table_width = table_coords[2] - table_coords[0]
        table_height = table_coords[3] - table_coords[1]
        margin_x = table_width * 0.2
        margin_y = table_height * 0.2

        expanded_table = [
            max(0, table_coords[0] - margin_x),
            max(0, table_coords[1] - margin_y),
            table_coords[2] + margin_x,
            table_coords[3] + margin_y
        ]

        for person in persons:
            # Method 1: IoU (overlap check)
            iou = calculate_iou(person, expanded_table)
            if iou > 0.01:
                is_occupied = True
                detection_method = "overlap"
                closest_distance = 0
                break

            # Method 2: Distance check
            distance = calculate_distance(person, table_coords)
            if distance < closest_distance:
                closest_distance = distance

            if distance < PROXIMITY_THRESHOLD:
                is_occupied = True
                detection_method = "proximity"
                break

        table_status[table_id] = {
            "id": table_id,
            "name": table["name"],
            "occupied": is_occupied,
            "coords": table_coords,
            "distance": round(closest_distance, 2) if closest_distance != float('inf') else None,
            "method": detection_method
        }

    return table_status

@app.get("/")
def read_root():
    return {"message": "FreeSpot API - Running on Google Colab", "status": "running"}

@app.get("/tables")
def get_tables():
    return {"tables": TABLES}

@app.post("/update-tables")
async def update_tables(data: dict):
    """Update konfigurasi meja dan proximity threshold"""
    global TABLES, PROXIMITY_THRESHOLD

    try:
        if "tables" in data:
            TABLES = data["tables"]

        if "proximity_threshold" in data:
            PROXIMITY_THRESHOLD = data["proximity_threshold"]

        return {
            "status": "success",
            "message": "Konfigurasi berhasil diupdate",
            "tables": TABLES,
            "proximity_threshold": PROXIMITY_THRESHOLD
        }
    except Exception as e:
        return JSONResponse({"error": str(e)}, status_code=500)

@app.post("/detect-frame")
async def detect_frame(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        nparr = np.frombuffer(contents, np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        results = model(frame, conf=0.5)

        persons = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                if int(box.cls[0]) == 0:
                    coords = box.xyxy[0].cpu().numpy()
                    persons.append(coords.tolist())

        table_status = check_table_occupancy(persons, TABLES)

        return JSONResponse({
            "table_status": list(table_status.values()),
            "persons_detected": len(persons),
            "timestamp": datetime.now().isoformat()
        })
    except Exception as e:
        return JSONResponse({"error": str(e)}, status_code=500)

@app.post("/detect-video")
async def detect_video(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        temp_input = "temp_input.mp4"
        temp_output = "temp_output.mp4"

        with open(temp_input, "wb") as f:
            f.write(contents)

        cap = cv2.VideoCapture(temp_input)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(temp_output, fourcc, fps, (width, height))

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame, conf=0.5)
            persons = []
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    if int(box.cls[0]) == 0:
                        coords = box.xyxy[0].cpu().numpy()
                        persons.append(coords.tolist())

            table_status = check_table_occupancy(persons, TABLES)
            annotated_frame = results[0].plot()

            for table in TABLES:
                coords = table["coords"]
                status = table_status[table["id"]]
                color = (0, 0, 255) if status["occupied"] else (0, 255, 0)
                cv2.rectangle(annotated_frame,
                             (coords[0], coords[1]),
                             (coords[2], coords[3]),
                             color, 3)
                cv2.putText(annotated_frame,
                           f"{table['name']}: {'Terpakai' if status['occupied'] else 'Kosong'}",
                           (coords[0], coords[1] - 10),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.6, color, 2)

            out.write(annotated_frame)
            frame_count += 1
            if frame_count % 30 == 0:
                print(f"Processed {frame_count} frames...")

        cap.release()
        out.release()
        print(f"✅ Video processing complete! Total frames: {frame_count}")

        def iterfile():
            with open(temp_output, mode="rb") as file_like:
                yield from file_like

        return StreamingResponse(iterfile(), media_type="video/mp4")
    except Exception as e:
        return JSONResponse({"error": str(e)}, status_code=500)

@app.post("/detect-video-stream")
async def detect_video_stream(file: UploadFile = File(...)):
    """
    Process video dengan SSE progress updates + generate video hasil deteksi
    """
    async def event_generator():
        temp_input = None
        temp_output = None
        video_id = str(uuid.uuid4())
        
        try:
            contents = await file.read()
            temp_input = f"temp_input_{video_id}.mp4"
            temp_output = f"temp_output_{video_id}.mp4"

            # Save input video
            with open(temp_input, "wb") as f:
                f.write(contents)

            # Open video for processing
            cap = cv2.VideoCapture(temp_input)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            frame_count = 0

            # Setup video writer
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(temp_output, fourcc, fps, (width, height))

            # Send start event
            yield {
                "event": "start",
                "data": json.dumps({
                    "status": "processing",
                    "total_frames": total_frames,
                    "fps": fps,
                    "video_id": video_id
                })
            }

            # Process frames
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1
                
                # YOLO detection
                results = model(frame, conf=0.5)
                persons = []
                for result in results:
                    boxes = result.boxes
                    for box in boxes:
                        if int(box.cls[0]) == 0:  # Person class
                            coords = box.xyxy[0].cpu().numpy()
                            persons.append(coords.tolist())

                # Check table occupancy
                table_status = check_table_occupancy(persons, TABLES)

                # Draw annotations on frame
                annotated_frame = results[0].plot()
                
                for table in TABLES:
                    coords = table["coords"]
                    status = table_status[table["id"]]
                    color = (0, 0, 255) if status["occupied"] else (0, 255, 0)
                    
                    # Draw table rectangle
                    cv2.rectangle(annotated_frame,
                                 (coords[0], coords[1]),
                                 (coords[2], coords[3]),
                                 color, 3)
                    
                    # Draw table label
                    cv2.putText(annotated_frame,
                               f"{table['name']}: {'Terpakai' if status['occupied'] else 'Kosong'}",
                               (coords[0], coords[1] - 10),
                               cv2.FONT_HERSHEY_SIMPLEX,
                               0.6, color, 2)
                
                # Write annotated frame to output video
                out.write(annotated_frame)

                # Send progress update
                if frame_count % 5 == 0 or frame_count == total_frames:
                    yield {
                        "event": "progress",
                        "data": json.dumps({
                            "frame": frame_count,
                            "total_frames": total_frames,
                            "progress": round((frame_count / total_frames) * 100, 2),
                            "table_status": list(table_status.values()),
                            "persons_detected": len(persons),
                            "timestamp": datetime.now().isoformat()
                        })
                    }

                await asyncio.sleep(0.01)

            # Release resources
            cap.release()
            out.release()

            # Store video info
            processed_videos[video_id] = {
                "path": temp_output,
                "timestamp": datetime.now().isoformat(),
                "frames": frame_count
            }

            # Send complete event with video URL
            yield {
                "event": "complete",
                "data": json.dumps({
                    "status": "completed",
                    "total_frames_processed": frame_count,
                    "video_id": video_id,
                    "video_url": f"/download-video/{video_id}"
                })
            }

            # Cleanup input file
            if temp_input and os.path.exists(temp_input):
                os.remove(temp_input)

        except Exception as e:
            yield {
                "event": "error",
                "data": json.dumps({
                    "error": str(e)
                })
            }
            
            # Cleanup on error
            if temp_input and os.path.exists(temp_input):
                os.remove(temp_input)
            if temp_output and os.path.exists(temp_output):
                os.remove(temp_output)

    return EventSourceResponse(event_generator())

@app.get("/download-video/{video_id}")
async def download_video(video_id: str):
    """Download processed video"""
    if video_id not in processed_videos:
        return JSONResponse({"error": "Video not found"}, status_code=404)
    
    video_info = processed_videos[video_id]
    video_path = video_info["path"]
    
    if not os.path.exists(video_path):
        return JSONResponse({"error": "Video file not found"}, status_code=404)
    
    return FileResponse(
        video_path,
        media_type="video/mp4",
        filename=f"freespot_result_{video_id}.mp4"
    )

active_connections: List[WebSocket] = []

@app.websocket("/ws/detect")
async def websocket_detect(websocket: WebSocket):
    await websocket.accept()
    active_connections.append(websocket)
    try:
        while True:
            data = await websocket.receive_bytes()
            nparr = np.frombuffer(data, np.uint8)
            frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

            results = model(frame, conf=0.5)
            persons = []
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    if int(box.cls[0]) == 0:
                        coords = box.xyxy[0].cpu().numpy()
                        persons.append(coords.tolist())

            table_status = check_table_occupancy(persons, TABLES)

            await websocket.send_json({
                "table_status": list(table_status.values()),
                "persons_detected": len(persons),
                "timestamp": datetime.now().isoformat()
            })
    except WebSocketDisconnect:
        active_connections.remove(websocket)

print("✅ Application configured successfully!")

## 🔄 Cell 4.5: Kill Old Server (Jika Ada)

**Jalankan cell ini jika mendapat error "address already in use"**

In [ ]:
# Kill proses yang menggunakan port 8000
import os
import signal
import subprocess

print("🔍 Checking for processes on port 8000...")

try:
    # Find process using port 8000
    result = subprocess.run(
        ["lsof", "-ti", ":8000"], 
        capture_output=True, 
        text=True
    )
    
    if result.stdout.strip():
        pids = result.stdout.strip().split('\n')
        print(f"⚠️  Found {len(pids)} process(es) using port 8000")
        
        for pid in pids:
            try:
                os.kill(int(pid), signal.SIGKILL)
                print(f"✅ Killed process PID: {pid}")
            except:
                print(f"❌ Failed to kill PID: {pid}")
        
        print("\n✅ Port 8000 is now free!")
    else:
        print("✅ Port 8000 is already free!")
        
except FileNotFoundError:
    # lsof might not be available, try alternative
    print("⚠️  lsof not found, trying fuser...")
    try:
        subprocess.run(["fuser", "-k", "8000/tcp"], check=False)
        print("✅ Port 8000 cleared using fuser!")
    except:
        print("⚠️  Could not clear port. Try: Runtime > Restart Runtime")

print("\n💡 Now you can run Cell 5 to start fresh server!")

## 🚀 Cell 5: Start Server

Jalankan cell ini untuk start server. 
**COPY URL** yang muncul di output dan gunakan di frontend!

In [ ]:
print("="*50)
print("🚀 Starting FreeSpot API Server...")
print("="*50)

# Start ngrok tunnel
public_url = ngrok.connect(8000)

print("\n✅ SERVER IS RUNNING!\n")
print("="*50)
print(f"🌐 Public URL: {public_url}")
print("="*50)
print("\n📋 INSTRUCTIONS:")
print(f"   1. Copy URL di atas: {public_url}")
print("   2. Di frontend, ganti 'http://localhost:8000' dengan URL tersebut")
print("   3. Untuk WebSocket, gunakan 'wss://' bukan 'ws://'")
print(f"\n🔗 API Documentation: {public_url}/docs")
print(f"🔗 Alternative Docs: {public_url}/redoc")
print("\n⚠️  CATATAN:")
print("   - Server akan berjalan sampai Anda stop cell ini (Runtime > Interrupt)")
print("   - Colab akan timeout setelah ~90 menit idle")
print("   - Setiap restart, URL akan berubah (kecuali pakai ngrok authtoken)")
print("\n" + "="*50)

# Start uvicorn server dengan asyncio (fix untuk Colab/Jupyter)
import asyncio
from threading import Thread

def run_server():
    """Run uvicorn server in background thread"""
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Start server in background thread
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("\n⏳ Waiting for server to start...")
import time
time.sleep(3)  # Wait for server to fully start

print("✅ Server started successfully!")
print("🔗 Test API: Send request to the public URL above")
print("\n💡 TIP: Keep this cell running. Don't interrupt it!")

# Keep cell running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n⏹️  Server stopped!")

---

## 📝 Notes & Usage Guide:

### 🎥 Video Detection Features:

#### 1. **Real-time Table Status Updates** (via SSE)
- Upload video → Lihat table status update per frame
- Progress bar menunjukkan persentase proses
- Table status grid update real-time

#### 2. **Video Hasil Deteksi** (Betav1.0 Logic)
- Setelah proses selesai, video hasil deteksi **otomatis tersedia**
- Video sudah ada **bounding box**:
  - 🟢 **Hijau** = Meja Kosong (EMPTY)
  - 🔴 **Merah** = Meja Terpakai (OCCUPIED)
- Bisa **ditonton langsung** di browser
- Bisa **didownload** dalam format MP4

#### 3. **Endpoints Available**:
```
POST /detect-video-stream  → Process video + SSE updates + generate video
GET  /download-video/{id}  → Download processed video
WS   /ws/detect            → Real-time webcam detection
```

### 🔧 Configuration:

#### Betav1.0 Detection Constants:
```python
CONFIDENCE_THRESHOLD = 0.25           # Detection confidence
DISTANCE_THRESHOLD_PIXELS = 120       # Person-to-table proximity
IOU_THRESHOLD = 0.45                  # Non-Maximum Suppression
DETECTION_INTERVAL = 5                # Re-detect tables every N frames
FRAME_OCCUPIED_THRESHOLD = 3          # Frames to mark occupied
FRAME_UNOCCUPIED_THRESHOLD = 8        # Frames to mark unoccupied
```

### 🚀 Untuk menggunakan model YOLO yang berbeda:
```python
model = YOLO('yolov11n.pt')  # Nano (fastest)
model = YOLO('yolov11s.pt')  # Small (balanced) - DEFAULT
model = YOLO('yolov11m.pt')  # Medium (more accurate)
model = YOLO('yolov11l.pt')  # Large (most accurate, slowest)
```

### ⚡ Untuk mengaktifkan GPU (HIGHLY RECOMMENDED):
1. Runtime > Change runtime type
2. Hardware accelerator > **GPU (T4)**
3. Restart Runtime
4. Re-run all cells

**Performance:**
- CPU: ~2-5 FPS (very slow)
- GPU T4: ~25-35 FPS (fast!)

### 🐛 Troubleshooting:

| Problem | Solution |
|---------|----------|
| **Error import** | Restart runtime dan run Cell 1 ulang |
| **Model download fail** | Check internet connection |
| **Ngrok URL expired** | Daftar dan gunakan authtoken di Cell 3 |
| **Out of memory** | Gunakan model lebih kecil (yolov11n.pt) |
| **Video tidak muncul** | Check browser console, pastikan ngrok URL benar |
| **Port 8000 in use** | Jalankan Cell 4.5 untuk kill old process |

### 📊 Frontend Setup:

**Update URL di `frontend/src/App.jsx`:**
```javascript
const API_URL = 'https://YOUR_NGROK_URL_HERE';  // Dari Cell 5
const WS_URL = 'wss://YOUR_NGROK_URL_HERE';     // Ganti ws:// dengan wss://
```

**Jalankan Frontend:**
```bash
cd frontend
npm install
npm run dev
```

### 🎯 Expected Workflow:

1. **User upload video** di frontend
2. **Backend process** dengan Betav1.0 logic:
   - Detect meja dinamis (YOLO "dining table")
   - Track per frame dengan NMS + IoU
   - Detect person proximity
   - Apply occupancy thresholds
3. **SSE updates** table status real-time ke frontend
4. **Generate video** dengan bounding box annotations
5. **Frontend display** video hasil deteksi
6. **User bisa download** video hasil

---

## 🎬 Video Output Details:

**Format:** MP4 (H.264)
**Resolution:** Same as input video
**FPS:** Same as input video
**Annotations:**
- Green rectangle + "EMPTY" text = Kosong
- Red rectangle + "OCCUPIED" text = Terpakai
- Rectangle drawn pada detected table position

---

Made with ❤️ using Betav1.0 Detection Logic + YOLO + FastAPI + Google Colab